<a href="https://colab.research.google.com/github/hyunicecream/Natural-Language-Processing-NLP-/blob/main/7_29_seq2seq(chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
# -*- coding: utf-8 -*-
"""7-3.seq2seq(chat).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1qMp24Tvn01XRtIx_ES_gVEqom99hm3SL
"""

# Commented out IPython magic to ensure Python compatibility.
# 작업 디렉토리를 변경한다.
# %cd '/content/drive/My Drive/Colab Notebooks'

# Seq2Seq 모델를 이용한 ChatBot : 채팅 모듈
#
# 관련 논문 : Kyunghyun Cho, et. al., 2014,
#            Learning Phrase Representations using RNN Encoder–Decoder 
#            for Statistical Machine Translation
#
# 저작자: 2021.05.26, 조성현 (blog.naver.com/chunjein)
# copyright: SNS 등에 공개할 때는 출처에 저작자를 명시해 주시기 바랍니다.
# ----------------------------------------------------------------------
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.layers import Embedding, TimeDistributed
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import sentencepiece as spm
import numpy as np
import pickle

In [ ]:
# Sub-word 사전 읽어온다.
with open('/content/drive/MyDrive/머신러닝/seq2seq/chatbot_voc.pkl', 'rb') as f:
    word2idx,  idx2word = pickle.load(f)

VOCAB_SIZE = len(idx2word)
EMB_SIZE = 128
LSTM_HIDDEN = 128
MAX_LEN = 15            # 단어 시퀀스 길이
MODEL_PATH = '/content/drive/MyDrive/머신러닝/seq2seq/chatbot_trained.h5'

# 데이터 전처리 과정에서 생성한 SentencePiece model을 불러온다.
SPM_MODEL = "/content/drive/MyDrive/머신러닝/seq2seq/chatbot_model.model"
sp = spm.SentencePieceProcessor()
sp.Load(SPM_MODEL)

# 워드 임베딩 레이어. Encoder와 decoder에서 공동으로 사용한다.
K.clear_session()
wordEmbedding = Embedding(input_dim=VOCAB_SIZE, output_dim=EMB_SIZE)

# Encoder
# -------
encoderX = Input(batch_shape=(None, MAX_LEN))
encEMB = wordEmbedding(encoderX)
encLSTM1 = LSTM(LSTM_HIDDEN, return_sequences=True, return_state = True)
encLSTM2 = LSTM(LSTM_HIDDEN, return_state = True)
ey1, eh1, ec1 = encLSTM1(encEMB)    # LSTM 1층 
_, eh2, ec2 = encLSTM2(ey1)         # LSTM 2층

# Decoder
# -------
# Decoder는 1개 단어씩을 입력으로 받는다.
decoderX = Input(batch_shape=(None, 1)) # 1 : time_step
decEMB = wordEmbedding(decoderX)
decLSTM1 = LSTM(LSTM_HIDDEN, return_sequences=True, return_state=True)
decLSTM2 = LSTM(LSTM_HIDDEN, return_sequences=True, return_state=True)
dy1, _, _ = decLSTM1(decEMB, initial_state = [eh1, ec1])
dy2, _, _ = decLSTM2(dy1, initial_state = [eh2, ec2])
decOutput = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))
outputY = decOutput(dy2)

# Model
# -----
model = Model([encoderX, decoderX], outputY)
model.load_weights(MODEL_PATH)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1152000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 15, 128), (N 131584      embedding[0][0]              

In [ ]:
# Chatting용 model
model_enc = Model(encoderX, [eh1, ec1, eh2, ec2]) # encoder 출력

# 채팅용 Decoder를 만든다
ih1 = Input(batch_shape = (None, LSTM_HIDDEN))
ic1 = Input(batch_shape = (None, LSTM_HIDDEN))
ih2 = Input(batch_shape = (None, LSTM_HIDDEN))
ic2 = Input(batch_shape = (None, LSTM_HIDDEN))

dec_output1, dh1, dc1 = decLSTM1(decEMB, initial_state = [ih1, ic1])
dec_output2, dh2, dc2 = decLSTM2(dec_output1, initial_state = [ih2, ic2])

dec_output = decOutput(dec_output2)
model_dec = Model([decoderX, ih1, ic1, ih2, ic2], [dec_output, dh1, dc1, dh2, dc2])

In [ ]:
import pdb
# Question을 입력받아 Answer를 생성한다.
def genAnswer(question): # 인코더 입력부분(1차원),  reshape
    question = question[np.newaxis, :] # 2차원 구조로 바꾸는 식 [np.newaxis, :] = question = qenstion.reshape(1, -1) -< (1x15) 행렬
    init_h1, init_c1, init_h2, init_c2 = model_enc.predict(question)

    # 시작 단어는 <BOS>로 한다.
    word = np.array(sp.bos_id()).reshape(1, 1)

    answer = []
    for i in range(MAX_LEN):
        dY, next_h1, next_c1, next_h2, next_c2 = model_dec.predict([word, init_h1, init_c1, init_h2, init_c2])
        # dY : 답변할 문장의 첫번째 subword
        # 디코더의 출력은 vocabulary에 대응되는 one-hot이다.
        # argmax로 해당 단어를 채택한다.
        nextWord = np.argmax(dY[0, 0])
        # np.random.mulitnomial(dY[0,0], 1) 
        # 융통성을 조절 할 수 있는 함수(장치) softmax

        # 예상 단어가 <EOS>이거나 <PAD>이면 더 이상 예상할 게 없다.       
        if nextWord == sp.eos_id() or nextWord == sp.pad_id():
            break
        
        # 다음 예상 단어인 디코더의 출력을 answer에 추가한다.
        answer.append(idx2word[nextWord])
        
        # 디코더의 다음 recurrent를 위해 입력 데이터와 hidden 값을
        # 준비한다. 입력은 word이고, hidden은 h와 c이다.
        word = np.array(nextWord).reshape(1,1)
    
        init_h1 = next_h1
        init_c1 = next_c1
        init_h2 = next_h2
        init_c2 = next_c2
        
    return sp.decode_pieces(answer) # 답변이 idx로 되어있어서 decode해야 한다. 
#pdb.set_trace()

In [ ]:
# Chatting
# dummy : 최초 1회는 모델을 로드하는데 약간의 시간이 걸리므로 이것을 가리기 위함.
def chatting(n=100):
    for i in range(n):
        question = input('Q : ')
        
        if  question == 'quit':
            break
        
        q_idx = []
        for x in sp.encode_as_pieces(question): # subword로 분해 
            if x in word2idx:
                q_idx.append(word2idx[x]) # word2idx로 id 부여
            else:
                q_idx.append(sp.unk_id())   # 사전에 없는 단어는 out-of-vocabulary (OOV) 처리
        
        # <PAD>를 삽입한다.
        if len(q_idx) < MAX_LEN:
            q_idx.extend([sp.pad_id()] * (MAX_LEN - len(q_idx)))
        else:
            q_idx = q_idx[0:MAX_LEN]
        
        answer = genAnswer(np.array(q_idx))
        print('A :', answer)

####### Chatting 시작 #######
print("\nSeq2Seq ChatBot (ver. 1.0)")
print("Chatting 모듈을 로드하고 있습니다 ...")

# 처음 1회는 시간이 걸리기 때문에 dummy question을 입력한다.
answer = genAnswer(np.zeros(MAX_LEN))
print("ChatBot이 준비 됐습니다.")



Seq2Seq ChatBot (ver. 1.0)
Chatting 모듈을 로드하고 있습니다 ...
ChatBot이 준비 됐습니다.


In [ ]:
# 채팅을 시작한다.
chatting(100)

Q : 김택현은 잘생겼니? 아니면 못생겼니?
A : 신경쓰이요
Q : 유현승은 얼짱이다
A : 그래도 늦지 않았어요
Q : 현승이는 잘생겼다
A : 네 말씀해주세요
Q : 쉣~
A : 좋은 사람 만나세요
Q : 가형님은 어떤사람?
A : 많이 더 더 더 힘들죠
Q : 지수님음 어떤사람?
A : 사랑은 항상 제일 생각하면서 알아차리지 알아차려도 싫어해주세요
Q : 지수님은 어떤사람?
A : 사랑은 항상 제일 알아보세요
Q : 이지수 + 진가형
A : 그게 더 나도 좋을 거예요
Q : 이지수 + 유현승 + 진가형
A : 그 사람의 변화를 따라게 좋아요
Q : 우리는 팀
A : 그게 더 좋은게요
Q : 가형님은 생각중
A : 좋은 생각을 항상 하세요
Q : 나는 좋은사람
A : 좋은 생각이에요
Q : 가형님은 고민중
A : 좋은 사람은 잘 될 거예요
Q : 현승이는 웃는중
A : 잘 잘 될 거예요
Q : 왓더뻑
A : 저도 좋죠
Q : quit


In [ ]:
# 대화 내용 예시
# 가끔 궁금해 : 그 사람도 그럴 거예요.,0
# 가끔 뭐하는지 궁금해 : 그 사람도 그럴 거예요.,0
# 가끔은 혼자인게 좋다 : 혼자를 즐기세요.,0
# 가난한 자의 설움 : 돈은 다시 들어올 거예요.,0
# 가만 있어도 땀난다 : 땀을 식혀주세요.,0
# 가상화폐 쫄딱 망함 : 어서 잊고 새출발 하세요.,0
# 가스불 켜고 나갔어 : 빨리 집에 돌아가서 끄고 나오세요.,0
# 가스불 켜놓고 나온거 같아 : 빨리 집에 돌아가서 끄고 나오세요.,0
# 가스비 너무 많이 나왔다. : 다음 달에는 더 절약해봐요.,0
# 가스비 비싼데 감기 걸리겠어 : 따뜻하게 사세요!,0
# 남자친구 교회 데려가고 싶어 : 마음을 열 때까지 설득해보세요.,0
# 남자친구 또 운동 갔어 : 운동을 함께 해보세요.,0
# 남자친구 생일인데 뭘 줄까 : 평소에 필요한 것 생각해보세요.,0
# 남자친구 승진 선물로 뭐가 좋을까? : 평소에 필요했던 게 좋을 것 같아요.,0
# 남자친구 오늘 따라 훈훈해 보인다 : 전생에 나라를 구하셨나요.,0
# 남자친구 오늘 좀 질린다. : 결단은 빠를수록 좋아요.,0
# 남자친구가 나 안 믿어줘 : 거짓말 적당히 하세요.,0
# 남자친구가 너무 바빠 : 너무 집착하지 마세요.,0
# 남자친구가 너무 운동만 해 : 운동을 함께 해보세요.,0
# 남자친구가 너무 잘생겼어 : 전생에 나라를 구하셨나요.,0

In [ ]:
# 융통성을 조절 할 수 있는 함수(장치) softmax
a = np.array([0.1, 0.5, 0.3, 0.4])
beta = 1.0

e = np.exp(a / beta)
e / np.sum(e)
# -> 배열이 확률 p로 변환 , 합은 = 1
# Mulitnomial : 1 과 3의 횟수 유사
# 융통성이 많다.
# beta가 클 수록 새로운 답변을 많이 하는 챗봇이 된다.

array([0.19750799, 0.2946473 , 0.24123681, 0.2666079 ])

In [ ]:
# 융통성을 조절 할 수 있는 함수(장치) softmax
a = np.array([0.1, 0.5, 0.3, 0.4])
beta = 0.1

e = np.exp(a / beta)
e / np.sum(e)
# -> 배열이 확률 p로 변환 , 합은 = 1
# 0.5와 0.4의 차이가 크다는 것을 확인 할 수 있다.
# Mulitnomial: 1 이 많이, 3이 작게
# 융통성이 작다. 
# beta가 작을 수록 답변이 딱딱해진다. 

array([0.01203764, 0.65723302, 0.08894682, 0.24178252])